<a href="https://colab.research.google.com/github/ShashankKamath/AutoML/blob/master/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import os
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import re

!pip install scikit-surprise
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
# from surprise import evaluate

import warnings; warnings.simplefilter('ignore')
os.chdir('/content/drive/My Drive/the-movies-dataset')

     |████████████████████████████████| 6.5MB 4.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678220 sha256=ec9a62511901ce7bdaab1e4cb1ed12f666bede00d390dbc98ff652007effe5b2
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


## Model 1 - Only Plot Description into consideration

In [36]:
# path = r'C:/Users/shash/PycharmProjects/Web-Scraping/Faculty Analysis/Dataset'
# os.chdir(r'C:\Users\shash\PycharmProjects\Web-Scraping\NLP\Project\dataset')
# Load Movies Metadata
metadata = pd.read_csv('movies_metadata.csv', low_memory=False)
# metadata['overview'].head()
# metadata.shape
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

#Output the shape of tfidf_matrix
# print(tfidf_matrix.shape)
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
#cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# sim_scores = linear_kernel(tfidf_matrix[idx], tfidf_matrix)
#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()
# Function that takes in movie title as input and outputs most similar movies

def get_recommendations(title):
    # Get the index of the movie that matches the title
    idx = indices[title]
    sim_scores = linear_kernel(tfidf_matrix[idx], tfidf_matrix)
    # Get the pairwsie similarity scores of all movies with that movie
    #sim_scores = list(enumerate(cosine_sim[idx]))
#    sim_scores = sim_scores[0]
#    sim_scores.sort()
    # Sort the movies based on the similarity scores
    sim_scores = list(enumerate(sim_scores[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]
    # print(recommeded_list)

    # for moviename in recommeded_list:
    #     print("Movie:",moviename)

    #     response = requests.get('https://api.themoviedb.org/3/search/movie?api_key=a6a79866abdec941dc00ef285bc96be3&query=' + moviename)
    #     # response = requests.get('https://api.themoviedb.org/3/movie/8844?api_key=a6a79866abdec941dc00ef285bc96be3')
    #     result_json=response.json()
    #     if len(result_json)>0:
    #         if len(result_json['results'])>0:
    #             image = requests.get('https://image.tmdb.org/t/p/w185/'+result_json['results'][0]['poster_path'])
    #             webbrowser.open_new_tab('https://image.tmdb.org/t/p/w185/'+result_json['results'][0]['poster_path'])
    #             # print(image)


# get_recommendations('The Dark Knight Rises')

print(get_recommendations('The Dark Knight Rises'))

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object


## Model 2 - Combining the cast, director, keywords, and plot 

In [0]:
credits = pd.read_csv("credits.csv")
keywords = pd.read_csv("keywords.csv")
metadata = pd.read_csv('movies_metadata.csv')

metadata = metadata.drop([19730, 29503, 35587])

keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merging the datasets
metadata = metadata.merge(credits, on="id")
metadata = metadata.merge(keywords, on="id")

Reducing the dataset size because of memory error

In [0]:
# C = metadata_1['vote_average'].mean()
# m = metadata_1['vote_count'].quantile(0.90)
# metadata = metadata_1.copy().loc[metadata_1['vote_count'] >= m]
# metadata.reset_index()

In [0]:
metadata['cast'] = metadata['cast'].apply(literal_eval)
metadata['crew'] = metadata['crew'].apply(literal_eval)
metadata['genres'] = metadata['genres'].apply(literal_eval)
metadata['keywords'] = metadata['keywords'].apply(literal_eval)
# metadata['cast_size'] = metadata['cast'].apply(lambda x: len(x))
# metadata['crew_size'] = metadata['crew'].apply(lambda x: len(x))

In [0]:
def get_director(x):
  for i in x:
      if i['job'] == 'Director':
          return i['name']
  return np.nan
metadata['director'] = metadata['crew'].apply(get_director)

In [0]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
  if isinstance(x, list):
      names = [i['name'] for i in x]
      #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
      if len(names) > 3:
          names = names[:3]
      return names

  #Return empty list in case of missing/malformed data
  return []

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [0]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
  if isinstance(x, list):
      return [str.lower(i.replace(" ", "")) for i in x]
  else:
      #Check if director exists. If not, return empty string
      if isinstance(x, str):
          return str.lower(x.replace(" ", ""))
      else:
          return ''

features = ['cast','director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [0]:
s = metadata.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s=s.value_counts()
s = s[s > 1]
stemmer = SnowballStemmer('english')


def filter_keywords(x):
  words = []
  for i in x:
      if i in s:
          words.append(i)
  return words

metadata['keywords'] = metadata['keywords'].apply(filter_keywords)
metadata['keywords'] = metadata['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
metadata['keywords'] = metadata['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [0]:
def scrub_words(text):
  """Basic cleaning of texts."""
  
  # remove html markup
  text=re.sub("(<.*?>)","",text)
  
  #remove non-ascii and digits
  text=re.sub("(\\W|\\d)"," ",text)
  
  #remove whitespace
  text=text.strip()
  return text

metadata['overview'] = metadata['overview'].fillna('')
metadata['overview'] = metadata['overview'].apply(scrub_words)
#Stemming the overview
stemmer = SnowballStemmer('english')
metadata['overview'] = metadata['overview'].apply(stemmer.stem)


In [0]:
# metadata[['title', 'cast', 'director', 'keywords', 'genres','overview']].head(3)

In [0]:
#Can add title and tagline as well
def create_soup(x):
  return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres']) + x['overview'] #+ x['title']
metadata['soup'] = metadata.apply(create_soup, axis=1)

In [0]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0,stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

from sklearn.metrics.pairwise import cosine_similarity

metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])


def get_recommendations(title,indices):
  idx = indices[title]
  sim_scores = cosine_similarity(count_matrix[idx], count_matrix)
  sim_scores = list(enumerate(sim_scores[0]))

  # Sort the movies based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Get the scores of the 10 most similar movies
  sim_scores = sim_scores[1:11]

  # Get the movie indices
  movie_indices = [i[0] for i in sim_scores]

  # Return the top 10 most similar movies
  return metadata['title'].iloc[movie_indices]

In [58]:
print(get_recommendations('The Dark Knight Rises', indices, cosine_sim2))

12589                            The Dark Knight
150                               Batman Forever
10210                              Batman Begins
585                                       Batman
18225                           Batman: Year One
1349                              Batman Returns
15682                 Batman: Under the Red Hood
19981    Batman: The Dark Knight Returns, Part 1
1511                              Batman & Robin
43147                      DC Showcase: Catwoman
Name: title, dtype: object


In [0]:
# metadata['cast'] = metadata['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
# metadata['cast'] = metadata['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
# metadata['cast'] = metadata['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
# metadata['keywords'] = metadata['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
# metadata['director'] = metadata['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
# metadata['director'] = metadata['director'].apply(lambda x: [x,x])

In [159]:
# metadata['soup'] = metadata['keywords'] + metadata['cast'] + metadata['director'] + metadata['genres'] + metadata['overview']
# metadata['soup'] = metadata['soup'].apply(lambda x: ' '.join(x))

# count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
# count_matrix = count.fit_transform(metadata['soup'])

# cosine_sim = cosine_similarity(count_matrix, count_matrix)

# metadata = metadata.reset_index()
# titles = metadata['title']
# indices = pd.Series(metadata.index, index=metadata['title'])

# def get_recommendations(title):
#     idx = indices[title]
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:31]
#     movie_indices = [i[0] for i in sim_scores]
#     return titles.iloc[movie_indices]

# get_recommendations('The Dark Knight Rises').head(10)

3174         The Dark Knight Rises
1984                 Batman Begins
47                  Batman Forever
3671              Reasonable Doubt
3141              Batman: Year One
2593      Mesrine: Killer Instinct
185                         Batman
2209                  The Prestige
2730           Law Abiding Citizen
2880    Batman: Under the Red Hood
Name: title, dtype: object

## Model 3 - Hybrid

In [15]:
reader = Reader()
ratings = pd.read_csv('ratings_small.csv')

#To have same split every time for testing
import random
random.seed(1)

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8968  0.8939  0.8934  0.9008  0.8985  0.8967  0.0028  
MAE (testset)     0.6901  0.6860  0.6891  0.6932  0.6929  0.6903  0.0026  
Fit time          4.59    4.52    4.62    4.61    4.54    4.58    0.04    
Test time         0.15    0.18    0.15    0.16    0.15    0.16    0.01    


{'fit_time': (4.5949084758758545,
  4.517142295837402,
  4.617350101470947,
  4.609635829925537,
  4.540955543518066),
 'test_mae': array([0.69009113, 0.68603977, 0.68905818, 0.69319   , 0.69291543]),
 'test_rmse': array([0.89681431, 0.89392165, 0.89337275, 0.90083335, 0.89848737]),
 'test_time': (0.1543419361114502,
  0.17890191078186035,
  0.15398502349853516,
  0.1576061248779297,
  0.15300250053405762)}

In [0]:
# Import seaborn library
import seaborn as sns
sns.set_style('whitegrid')
sns.set(font_scale=1.5)
%matplotlib inline

# Display distribution of rating
sns.distplot(ratings['rating'].fillna(ratings['rating'].median()))

In [0]:
def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan

In [0]:
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(metadata[['title', 'id']], on='id').set_index('title')

In [0]:
indices_map = id_map.set_index('id')
metadata = metadata.reset_index()

In [0]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [0]:
def check_index(x):
  if x in indices_map:
    return svd.predict(userId, indices_map.loc[x]['movieId']).est
  else: 
    pass

def hybrid(userId, title):
  idx = indices[title]
  # tmdbId = id_map.loc[title]['id']
  #print(idx)
  movie_id = id_map.loc[title]['movieId']
  
  sim_scores = cosine_similarity(count_matrix[idx], count_matrix)
  sim_scores = list(enumerate(sim_scores[0]))
  # sim_scores = list(enumerate(cosine_sim[int(idx)]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:26]
  movie_indices = [i[0] for i in sim_scores]
  # print(movie_indices)

  movies = metadata.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
  # print(movies.head(50))

  # vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
  # vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
  # C = vote_averages.mean()
  # m = vote_counts.quantile(0.60)
  # qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
  # qualified['vote_count'] = qualified['vote_count'].astype('int')
  # qualified['vote_average'] = qualified['vote_average'].astype('int')
  # qualified['wr'] = qualified.apply(weighted_rating, axis=1)
  # qualified = qualified.sort_values('wr', ascending=False).head(25)
  
  movies['est'] = movies['id'].apply(check_index)
  # movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
  movies = movies.sort_values('est', ascending=False)
  return movies.head(10)

In [108]:
hybrid(1, 'The Dark Knight')

,title,vote_count,vote_average,id,est
18442,The Dark Knight Rises,9263.0,7.6,49026,None
10210,Batman Begins,7511.0,7.5,272,None
150,Batman Forever,1529.0,5.2,414,None
35011,İtirazım Var,9.0,7.1,265351,None
25575,Gangsters,0.0,0.0,78566,None
32471,Joni,0.0,0.0,124523,None
6111,Q & A,22.0,6.6,31598,None
24066,In Order of Disappearance,142.0,7.0,252822,None
30092,Accused,30.0,6.6,242065,None
5464,Love Me Tender,21.0,5.5,39833,None


In [109]:
hybrid(500, 'The Dark Knight Rises')

,title,vote_count,vote_average,id,est
12589,The Dark Knight,12269.0,8.3,155,None
150,Batman Forever,1529.0,5.2,414,None
10210,Batman Begins,7511.0,7.5,272,None
585,Batman,2145.0,7.0,268,None
18225,Batman: Year One,255.0,7.1,69735,None
1349,Batman Returns,1706.0,6.6,364,None
15682,Batman: Under the Red Hood,459.0,7.6,40662,None
19981,"Batman: The Dark Knight Returns, Part 1",410.0,7.7,123025,None
1511,Batman & Robin,1447.0,4.2,415,None
43147,DC Showcase: Catwoman,36.0,6.5,76420,None


## Neural Network Approach - Waste


In [0]:
import numpy as np
import pandas as pd
import os
import sys
import pickle
import time
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from importlib import reload
import torch
%matplotlib inline

### Tf-IDF generation

In [9]:
ratings = pd.read_csv('ratings_small.csv')
ratings = ratings.drop_duplicates('movieId')
print("{} unique movies in ratings.csv".format(len(ratings.movieId.unique())))

9066 unique movies in ratings.csv


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    ngram_range=(0, 1),
    min_df=0.0001,
    stop_words='english')
tfidf_matrix = tfidf.fit_transform(metadata['soup'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=metadata.index.tolist())
# print(tfidf_df.shape)
# tfidf_df.to_pickle('tfidf_matrix.pkl')
# tfidf_df.head(3)

### Auto Encoder


In [0]:
from autoencoder import AutoEncoder
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pickle

import multiprocessing

import torch.nn as nn
import torch.optim as optim

In [0]:
ae = AutoEncoder(tfidf_df, validation_perc=0.1, lr=1e-3, intermediate_size=5000, encoded_size=100)
ae.train_loop(30)

In [0]:
losses = pd.DataFrame(data=list(zip(ae.train_losses, ae.val_losses)), columns=['train_loss', 'validation_loss'])
losses['epoch'] = (losses.index + 1) / 3

In [0]:
%config InlineBackend.figure_format = 'svg'
fig, ax = plt.subplots()
ax.plot(losses['epoch'], losses['train_loss'])
ax.plot(losses['epoch'], losses['validation_loss'])
ax.set_ylabel('MSE loss')
ax.set_xlabel('epoch')
ax.set_title('autoencoder loss over time')
ax.legend()

In [0]:
encoded = ae.get_encoded_representations()
with open('autoencoder_embeddings.pkl', 'wb') as fh:
    pickle.dump(encoded, fh)

## Word2Vec Method

In [0]:
credits = pd.read_csv("credits.csv")
keywords = pd.read_csv("keywords.csv")
metadata = pd.read_csv('movies_metadata.csv')

metadata = metadata.drop([19730, 29503, 35587])

keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merging the datasets
metadata = metadata.merge(credits, on="id")
metadata = metadata.merge(keywords, on="id")

metadata['cast'] = metadata['cast'].apply(literal_eval)
metadata['crew'] = metadata['crew'].apply(literal_eval)
metadata['genres'] = metadata['genres'].apply(literal_eval)
metadata['keywords'] = metadata['keywords'].apply(literal_eval)

def get_director(x):
  for i in x:
      if i['job'] == 'Director':
          return i['name']
  return np.nan
metadata['director'] = metadata['crew'].apply(get_director)

In [0]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
  if isinstance(x, list):
      names = [i['name'] for i in x]
      #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
      if len(names) > 3:
          names = names[:3]
      return names

  #Return empty list in case of missing/malformed data
  return []

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [0]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
  if isinstance(x, list):
      return [str.lower(i.replace(" ", "")) for i in x]
  else:
      #Check if director exists. If not, return empty string
      if isinstance(x, str):
          return str.lower(x.replace(" ", ""))
      else:
          return ''

features = ['cast','director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [0]:
s = metadata.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s=s.value_counts()
s = s[s > 1]
stemmer = SnowballStemmer('english')


def filter_keywords(x):
  words = []
  for i in x:
      if i in s:
          words.append(i)
  return words

metadata['keywords'] = metadata['keywords'].apply(filter_keywords)
metadata['keywords'] = metadata['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
metadata['keywords'] = metadata['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [0]:
def scrub_words(text):
  """Basic cleaning of texts."""
  
  # remove html markup
  text=re.sub("(<.*?>)","",text)
  
  #remove non-ascii and digits
  text=re.sub("(\\W|\\d)"," ",text)
  
  #remove whitespace
  text=text.strip()
  return text

metadata['overview'] = metadata['overview'].fillna('')
metadata['overview'] = metadata['overview'].apply(scrub_words)
#Stemming the overview
stemmer = SnowballStemmer('english')
metadata['overview'] = metadata['overview'].apply(stemmer.stem)


In [0]:
import string
metadata['tagline'] = metadata['tagline'].fillna('')
metadata['title'] = metadata['title'].fillna('')
def clean_tag_title(text):
  text=text.strip()
  text=text.lower()
  text = text.translate(string.punctuation)
  return text
metadata['tagline'] = metadata['tagline'].apply(clean_tag_title)
# metadata['title'] = metadata['title'].apply(clean_tag_title)

In [0]:
#Can add title and tagline as well
def create_soup(x):
  return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres']) +' '+ x['overview'] + ' ' + x['title'] +' '+ x['tagline']
metadata['soup'] = metadata.apply(create_soup, axis=1)

def clean_soup(text):
  text=text.strip()
  text=text.lower()
  text = text.translate(string.punctuation)
  return text
metadata['soup'] = metadata['soup'].apply(clean_soup)

In [11]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

from nltk.tokenize import word_tokenize
def apply_token(text):
  tokens = word_tokenize(text)
  result = [i for i in tokens if not i in stop_words]
  return result
metadata['soup'] = metadata['soup'].apply(apply_token)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
from collections import defaultdict
class TfidfEmbeddingVectorizer(object):
  def __init__(self, word2vec):
    self.word2vec = word2vec
    self.word2weight = None
    if len(word2vec)>0:
        self.dim=len(word2vec[next(iter(w2v))])
    else:
        self.dim=0
      
  def fit(self, X, y):
    tfidf = TfidfVectorizer(analyzer=lambda x: x)
    tfidf.fit(X)
    # if a word was never seen - it must be at least as infrequent
    # as any of the known words - so the default idf is the max of 
    # known idf's
    max_idf = max(tfidf.idf_)
    self.word2weight = defaultdict(lambda: max_idf, [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    return self
  
  def transform(self, X):
    return np.array([np.mean([self.word2vec[w] * self.word2weight[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0) for words in X])

In [0]:
import gensim
# let X be a list of tokenized texts (i.e. list of lists of tokens)
X = metadata['soup'].tolist()
model = gensim.models.Word2Vec(X, size=100)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [0]:
count = TfidfEmbeddingVectorizer(w2v)
count = count.fit(metadata['soup'],None)
count_matrix = count.transform(metadata['soup'])

from sklearn.metrics.pairwise import cosine_similarity
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [0]:
def get_recommendations(title):
  # title=title.lower()
  idx = indices[title]
  print("Movie Index",idx)
  sim_scores = cosine_similarity(count_matrix[idx].reshape(1,-1), count_matrix)
  sim_scores = list(enumerate(sim_scores[0]))

  # Sort the movies based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Get the scores of the 10 most similar movies
  sim_scores = sim_scores[1:11]

  # Get the movie indices
  movie_indices = [i[0] for i in sim_scores]
  print(movie_indices)

  # Return the top 10 most similar movies
  return metadata['title'].iloc[movie_indices],metadata['poster_path'].iloc[movie_indices]

In [32]:
idx = indices['The Dark Knight'].iloc[0]
print(idx)

12589


In [34]:
a,b = get_recommendations('The Dark Knight')
print(a)
print(b)

Movie Index 12589
[26629, 8122, 28208, 20476, 8914, 5409, 18442, 41288, 8897, 10438]
26629                             Wicked Blood
8122     Sherlock Holmes and the Secret Weapon
28208             The Three Fantastic Supermen
20476                              Broken City
8914                          The Star Chamber
5409                            Modesty Blaise
18442                    The Dark Knight Rises
41288                                   Race 2
8897                              Blue Thunder
10438                     Revenge of the Ninja
Name: title, dtype: object
26629    /1wbeqDYxZDmm2Gdg3vDdKiJFi00.jpg
8122     /mJyEiDQrxsHlH6oQPeZy4MQ78Rm.jpg
28208    /iz9uLv2meeYmcQO0KkoA0jI1wu6.jpg
20476    /pYDj2b15p4YdNz88XvO7QmBoHgD.jpg
8914     /kBXvrwS0gptAVZ8JkVFVylPxJh8.jpg
5409     /b6RMnkos9UltmeOb4h6g2xzlpPt.jpg
18442    /dEYnvnUfXrqvqeRSqvIEtmzhoA8.jpg
41288    /3xXL3VNUNfl6XZQQcTbMLaKirs4.jpg
8897     /sdsA2shw0TA81idpx2EafI6FEwN.jpg
10438    /hhVQongUUKCSwZG0RqVIRKVnYVE.jp

In [64]:
print(get_recommendations('The Dark Knight Rises'))

18442
[12589, 26629, 8221, 437, 18863, 1920, 28783, 4112, 13114, 11123]
12589                                    the dark knight
26629                                       wicked blood
8221               sky captain and the world of tomorrow
437                                       demolition man
18863                            mr. moto's last warning
1920                                       soylent green
28783    ghost in the shell arise - border 1: ghost pain
4112                                   death on the nile
13114                                          max payne
11123                                      hollow man ii
Name: title, dtype: object
